# Final project for Introduction to Quantum Computing (18-819F) Quantum Solver

Project Description:
- Scalable and Accurate Generation of Hybrid MPC Protocols with Quantum Integer Programming

## Preparation

### Bootstrap

In [ ]:
import numpy as np
import time
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp

# Initialize the account first.
service = QiskitRuntimeService()

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

### Available Backends

In [ ]:
service.backends()

### SILPH Reader

In [ ]:
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr

def parse_problem(fname):
    mdl = Model(fname)
    
    # Term Variable Name to Variable 
    # {name: (variable, cost)}
    term_var = {}
    # Conv Variable Name to Variable 
    # {name: (variable, cost)}
    conv_var = {}
    for line in open(fname):
        codes = line.split()
        if codes[0] == "VT":
            # term var
            if not codes[1] in term_var:
                v = mdl.binary_var(codes[1])
                term_var[codes[1]] = (v, float(codes[2]))
        elif codes[0] == "VC":
            # conversion var
            if not codes[1] in conv_var:
                v = mdl.binary_var(codes[1])
                conv_var[codes[1]] = (v, float(codes[2]))
            
        elif codes[0] == "CA":
            # assignment constraint
            if len(codes) == 2:
                # CA term_var
                (v1, _) = term_var[codes[1]]
                mdl.add_constraint(v1 >= 1)
            elif len(codes) == 3:
                # CA term_var term_var
                (v1, _) = term_var[codes[1]]
                (v2, _) = term_var[codes[2]]
                mdl.add_constraint(v1 + v2 >= 1)
            elif len(codes) == 4:
                # CA term_var term_var term_var
                (v1, _) = term_var[codes[1]]
                (v2, _) = term_var[codes[2]]
                (v3, _) = term_var[codes[3]]
                mdl.add_constraint(v1 + v2 + v3 >= 1)
            
        elif codes[0] == "CC":
            # conversion constraint
            (v1, _) = conv_var[codes[1]]
            (v2, _) = term_var[codes[2]]
            (v3, _) = term_var[codes[3]]
            mdl.add_constraint(v1 >= v2 + v3 - 1)
            
    # Create objective funciton
    exp = mdl.linear_expr()
    for (v, cost) in term_var.values():
        exp.add(v*cost)
    for (v, cost) in conv_var.values():
        exp.add(v*cost)
    mdl.minimize(exp)
    return mdl
        

### QUBO converter

In [ ]:
from qiskit_optimization.converters import InequalityToEquality
from qiskit_optimization.converters import IntegerToBinary
from qiskit_optimization.converters import LinearEqualityToPenalty

In [ ]:
ineq2eq = InequalityToEquality()
int2bin = IntegerToBinary()
lineq2penalty = LinearEqualityToPenalty()
def to_qubo(qp):
    qp_eq = ineq2eq.convert(qp)
    qp_eq_bin = int2bin.convert(qp_eq)
    qubo = lineq2penalty.convert(qp_eq_bin)
    return qubo
    

#### Convert to Ising

To Ising

In [ ]:
# qubitOp, offset = qubo.to_ising()
# print("Offset:", offset)
# print("Ising Hamiltonian:")
# print(str(qubitOp))

### Test case

In [ ]:
# mdl = parse_problem("./toy_1.txt")
mdl = parse_problem("./toy_2.txt")
# mdl = parse_problem("./biomatch_4.txt")
# mdl = parse_problem("./biomatch_16.txt")
# mdl = parse_problem("./kmeans.txt")

# print(mdl.export_as_lp_string())
qp = from_docplex_mp(mdl)
qubo = to_qubo(qp)

## Annealing

In [ ]:
!pip install dwave-ocean-sdk --quiet

In [ ]:
import dimod
import neal

import matplotlib.pyplot as plt
import numpy as np
from scipy.special import gamma
import math
from collections import Counter
import pandas as pd
from itertools import chain
import time
import networkx as nx

In [ ]:
# qiskit qubo to dwave bqm
linear = qubo.objective.linear.to_dict()
quadratic = qubo.objective.quadratic.to_dict()
constant = qubo.objective.constant
model = dimod.BinaryQuadraticModel(linear, quadratic, constant, dimod.Vartype.BINARY)

In [ ]:
print(model)

### Graphing

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def plot_enumerate(results, fig_name, title=None):

    plt.figure()

    energies = [datum.energy for datum in results.data(
        ['energy'], sorted_by=None)]
    
    if results.vartype == 'Vartype.BINARY':
        samples = [''.join(c for c in str(datum.sample.values()).strip(
            ', ') if c.isdigit()) for datum in results.data(['sample'], sorted_by=None)]
        plt.xlabel('bitstring for solution')
    else:
        samples = np.arange(len(energies))
        plt.xlabel('solution')

    plt.bar(samples,energies, color=(0.2, 0.4, 0.6, 0.6))
    plt.xticks(rotation=90)
    plt.ylabel('Energy')
    plt.title(str(title))
    print("minimum energy:", min(energies))
    plt.savefig(fig_name)


def plot_energies(results, title=None):
    energies = results.data_vectors['energy']
    occurrences = results.data_vectors['num_occurrences']
    counts = Counter(energies)
    total = sum(occurrences)
    counts = {}
    for index, energy in enumerate(energies):
        if energy in counts.keys():
            counts[energy] += occurrences[index]
        else:
            counts[energy] = occurrences[index]
    for key in counts:
        counts[key] /= total
    df = pd.DataFrame.from_dict(counts, orient='index').sort_index()
    df.plot(kind='bar', legend=None)

    plt.xlabel('Energy')
    plt.ylabel('Probabilities')
    plt.title(str(title))
    plt.show()
    print("minimum energy:", min(energies))

def parse_energy(results, name):
    energies = results.data_vectors['energy']
    occurrences = results.data_vectors['num_occurrences']
    counts = Counter(energies)
    total = sum(occurrences)
    counts = {}
    for index, energy in enumerate(energies):
        if energy in counts.keys():
            counts[energy] = [energy, counts[energy][1] + occurrences[index], name]
        else:
            counts[energy] = [energy, occurrences[index], name]
    return counts

def plot_density(result1, result2):
    counts_1 = parse_energy(result1, "Simulated Annealing")
    counts_2 = parse_energy(result2, "Quantum Annealing")
    plt.figure(figsize=[8, 5])
    df_1 = pd.DataFrame.from_dict(counts_1, orient='index').sort_index()
    df_2 = pd.DataFrame.from_dict(counts_2, orient='index').sort_index()
    df = pd.concat([df_1, df_2])
    sns.set(style="whitegrid", color_codes=True)
    ax = sns.stripplot(
        data=df,
        x=2, y=0, size=2, jitter=0.2
    )
    ax.set(xlabel='Methods', ylabel='Energy')
    plt.savefig("./graph_quantum_annealing_1000_toy_2.pdf")

    
    

### Simulated Annealing

In [ ]:
t0 = time.time()

sweeps_at_beta = 1
simAnnSampler = neal.SimulatedAnnealingSampler()
simAnnSamples = simAnnSampler.sample(
    model, 
    # beta_schedule_type="custom", 
    # # initial_states = np.zeros(len(model.variables), dtype=np.int8), 
    # num_sweeps_per_beta = 10,
    # beta_schedule = np.linspace(0.01, 10, num=1000),
    # num_sweeps=10000,
    # initial_states_generator="tile",
    # seed=12345,
    num_reads=1000)
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
plot_enumerate(simAnnSamples, "./graph_enumerate_simulated_annealing_1000_toy_2.pdf", title='Simulated annealing in default parameters')

In [ ]:
print(simAnnSamples.aggregate().lowest())

### Quantum Annealing

In [ ]:
!echo -e "\n" | dwave setup -a

In [ ]:
import dwave_networkx as dnx
from dwave.system import (DWaveSampler, EmbeddingComposite,
                          FixedEmbeddingComposite)
from pprint import pprint

In [ ]:
!dwave solvers  --list --all

In [ ]:
!dwave ping

In [ ]:
qpu = DWaveSampler()
qpu_edges = qpu.edgelist
qpu_nodes = qpu.nodelist
# pprint(dir(qpu))
print(qpu.solver.id)
X = dnx.pegasus_graph(16, node_list=qpu_nodes, edge_list=qpu_edges)
dnx.draw_pegasus(X, node_size=1)
print('Number of qubits=', len(qpu_nodes))
print('Number of couplers=', len(qpu_edges))

In [ ]:
DWavesampler = EmbeddingComposite(DWaveSampler())
t0 = time.time()
DWaveSamples = DWavesampler.sample(bqm=model, num_reads=1000, 
                                   return_embedding=True, 
                                  #  chain_strength=chain_strength, 
                                   # annealing_time=100
                                   )
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
# print(DWaveSamples.info)

In [ ]:
plot_enumerate(DWaveSamples, "./graph_enumerate_quantum_annealing_1000_toy_2.pdf", title='Quantum annealing in default parameters')

In [ ]:
print(DWaveSamples.aggregate().lowest())

#### Comparison

In [ ]:
plot_density(simAnnSamples, DWaveSamples)

In [ ]:
chain_strength = 1000

In [ ]:
t0 = time.time()
DWaveSamples = DWavesampler.sample(bqm=model, num_reads=1000, 
                                   annealing_time=1
                                   )
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
t0 = time.time()
DWaveSamples2 = DWavesampler.sample(bqm=model, num_reads=1000, 
                                   annealing_time=10
                                   )
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
t0 = time.time()
DWaveSamples3 = DWavesampler.sample(bqm=model, num_reads=1000, 
                                   annealing_time=50
                                   )
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
t0 = time.time()
DWaveSamples4 = DWavesampler.sample(bqm=model, num_reads=1000, 
                                   annealing_time=100
                                   )
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
t0 = time.time()
DWaveSamples5 = DWavesampler.sample(bqm=model, num_reads=1000, 
                                   annealing_time=500
                                   )
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
t0 = time.time()
DWaveSamples6 = DWavesampler.sample(bqm=model, num_reads=1000, 
                                   annealing_time=750
                                   )
t1 = time.time()
print("timing:", t1-t0)

In [ ]:
counts_1 = parse_energy(DWaveSamples, "1")
counts_2 = parse_energy(DWaveSamples2, "10")
counts_3 = parse_energy(DWaveSamples3, "50")
counts_4 = parse_energy(DWaveSamples4, "100")
counts_5 = parse_energy(DWaveSamples5, "500")
counts_6 = parse_energy(DWaveSamples6, "750")
plt.figure(figsize=[8, 5])
df_1 = pd.DataFrame.from_dict(counts_1, orient='index').sort_index()
df_2 = pd.DataFrame.from_dict(counts_2, orient='index').sort_index()
df_3 = pd.DataFrame.from_dict(counts_3, orient='index').sort_index()
df_4 = pd.DataFrame.from_dict(counts_4, orient='index').sort_index()
df_5 = pd.DataFrame.from_dict(counts_5, orient='index').sort_index()
df_6 = pd.DataFrame.from_dict(counts_6, orient='index').sort_index()
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6])
sns.set(style="whitegrid", color_codes=True)
ax = sns.stripplot(
    data=df,
    x=2, y=0, size=2, jitter=0.2
)
ax.set(xlabel='Anneal time(num_reads = 1000)', ylabel='Energy')
plt.savefig("./graph_quantum_annealing_opt_anneal_time.pdf")


In [ ]:
print(DWaveSamples.aggregate().lowest())
print(DWaveSamples2.aggregate().lowest())
print(DWaveSamples3.aggregate().lowest())
print(DWaveSamples4.aggregate().lowest())
print(DWaveSamples5.aggregate().lowest())
print(DWaveSamples6.aggregate().lowest())

In [ ]:
from hybrid.reference.kerberos import KerberosSampler

In [ ]:
# mdl = parse_problem("./toy_1.txt")
# mdl = parse_problem("./toy_2.txt")
# mdl = parse_problem("./biomatch_4.txt")
mdl = parse_problem("./biomatch_16.txt")
# mdl = parse_problem("./kmeans.txt")

# print(mdl.export_as_lp_string())
qp = from_docplex_mp(mdl)
qubo = to_qubo(qp)
# qiskit qubo to dwave bqm
linear = qubo.objective.linear.to_dict()
quadratic = qubo.objective.quadratic.to_dict()
constant = qubo.objective.constant
model = dimod.BinaryQuadraticModel(linear, quadratic, constant, dimod.Vartype.BINARY)

In [ ]:
DWavesampler = EmbeddingComposite(DWaveSampler())
t0 = time.time()
solution = KerberosSampler().sample(model, max_iter=100, convergence=5, qpu_sampler=DWavesampler)
t1 = time.time()
print("timing:", t1-t0)
print("result:", solution)

#### HybridSolver

In [ ]:
from dwave.system import LeapHybridSampler

In [ ]:
sampler = LeapHybridSampler()  
t0 = time.time()
sampleset = sampler.sample(model) 
t1 = time.time()
print("timing:", t1-t0)
print("result:", sampleset)